In [38]:
#https://qwen.readthedocs.io/en/latest/inference/chat.html
#https://www.philschmid.de/fine-tune-llms-in-2024-with-trl#3-create-and-prepare-the-dataset

In [28]:
# https://explainshell.com/explain?cmd=tree+.+--charset+utf-8+-d+-L+2

#!tree . --charset utf-8 -d -L 2
!tree --charset utf-8 -d -L 2

.
├── commonsense_qa
│   ├── test
│   ├── train
│   └── validation
├── models
│   ├── Qwen2-0.5B
│   ├── Qwen2-7B
│   ├── Qwen2.5-0.5B
│   └── Qwen2.5-0.5B-Instruct
├── qwen2.5:0.5B-Cot0-val
├── qwen2.5:0.5B-Instruct-Cot0-val
├── qwen2:0.5B-Cot0-val
├── qwen2:0.5B-Cot0optimised1-val
├── qwen2:0.5B-Cot0optimised2-val
├── qwen2:0.5B-icl5-val
├── qwen2:0.5B-icl8-val
├── qwen2:0.5B-standard-val
└── qwen2:7B-standard-val

18 directories


In [2]:
from transformers import pipeline
from datasets import load_dataset, load_from_disk
import os

In [6]:
# load model
model_name = "Qwen2-0.5B"
#model_name = "Qwen2-7B"

models_dir = "models"
if not os.path.exists(models_dir):
    os.mkdir(models_dir)

if not os.path.exists(os.path.join(models_dir,model_name)):
    pipe = pipeline("text-generation", f"Qwen/{model_name}", torch_dtype="auto", device_map="auto")
    pipe.tokenizer.padding_side="left"
    pipe.save_pretrained(os.path.join(models_dir,model_name))

else:
    print("Load from disk")
    pipe = pipeline("text-generation", os.path.join(models_dir,model_name), torch_dtype="auto", device_map="auto")
    pipe.tokenizer.padding_side="left"


Load from disk


In [7]:
def send_prompt_to_LLM(prompt):
    
    messages = [{"role": "user", "content": prompt}]
    response_message = pipe(messages, max_new_tokens=512)[0]["generated_text"][-1]
    #messages.append({"role": "user", "content": prompt})
    #response_message = pipe(messages, max_new_tokens=512)[0]["generated_text"][-1]
    
    response_message = response_message['content']
    return response_message

In [8]:
# load dataset
dataset_name_hub = "tau/commonsense_qa"
dataset_local_dir = "commonsense_qa"

if not os.path.exists(dataset_local_dir):
    commonsenseQA = load_dataset(dataset_name_hub)
    commonsenseQA.save_to_disk(dataset_local_dir)
else:
    print("Load from disk")
    commonsenseQA = load_from_disk(dataset_local_dir)

Load from disk


In [9]:
for i,row in enumerate(commonsenseQA['validation']):
    if i == 8:
        break
    print(row)
    print("\n")

{'id': '1afa02df02c908a558b4036e80242fac', 'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?', 'question_concept': 'revolving door', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['bank', 'library', 'department store', 'mall', 'new york']}, 'answerKey': 'A'}


{'id': 'a7ab086045575bb497933726e4e6ad28', 'question': 'What do people aim to do at work?', 'question_concept': 'people', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']}, 'answerKey': 'A'}


{'id': 'b8c0a4703079cf661d7261a60a1bcbff', 'question': 'Where would you find magazines along side many other printed works?', 'question_concept': 'magazines', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['doctor', 'bookstore', 'market', 'train station', 'mortuary']}, 'answerKey': 'B'}


{'id': 'e68fb2448fd74e402aae9982aa76e527', 'question': 'Where ar

## **Prompts**

#### **Standard**

In [10]:
def create_standard_prompt(row):
    
    question_para = f"\nQ: {row['question']}"
    
    options_para = '\noptions: '
    for label, text in zip(row['choices']['label'], row['choices']['text']):
        options_para = options_para + f" ({label}) {text}"
        
    prompt =  f"Answer the following multiple choice question: {question_para} {options_para}"
    return prompt

print(create_standard_prompt(row))

Answer the following multiple choice question: 
Q: Reading newspaper one of many ways to practice your what? 
options:  (A) literacy (B) knowing how to read (C) money (D) buying (E) money bank


#### **Chain-of_Thought 0 shot**

In [11]:
def create_CoT0shot_prompt(row):
    
    question_para = f"\nQ: {row['question']}"
    
    options_para = '\noptions: '
    for label, text in zip(row['choices']['label'], row['choices']['text']):
        options_para = options_para + f" ({label}) {text}"
        
    CoT_para = f"\n\nLet's think step by step before arriving at the final answer."
    
    prompt =  f"Answer the following multiple choice question: \n{question_para} {options_para} {CoT_para}"
    return prompt

print(create_CoT0shot_prompt(row))

Answer the following multiple choice question: 

Q: Reading newspaper one of many ways to practice your what? 
options:  (A) literacy (B) knowing how to read (C) money (D) buying (E) money bank 

Let's think step by step before arriving at the final answer.


#### **Chain-of_Thought 0 shot optimised**

In [12]:
def create_CoT0shotoptimised1_prompt(row):
    
    question_para = f"\nQ: {row['question']}"
    
    options_para = '\noptions: '
    for label, text in zip(row['choices']['label'], row['choices']['text']):
        options_para = options_para + f" ({label}) {text}"
        
    CoT_para = f"\n\nLet's think step by step before arriving at the final answer. "
    CoT_para =  CoT_para + f"\nStep 1. Consider each option given one by one, for each option assume the option is the answer to the question, and use the option to answer the question."
    CoT_para =  CoT_para + f"\nStep 2. Pick which one of the 5 answer makes the most sense."
    CoT_para =  CoT_para + f"\nStep 3. Answer the mutiple choice question with the previous insights from Step 1 and Step 2."
    
    prompt =  f"Answer the following multiple choice question: \n{question_para} {options_para} {CoT_para}"
    return prompt

print(create_CoT0shotoptimised1_prompt(row))

Answer the following multiple choice question: 

Q: Reading newspaper one of many ways to practice your what? 
options:  (A) literacy (B) knowing how to read (C) money (D) buying (E) money bank 

Let's think step by step before arriving at the final answer. 
Step 1. Consider each option given one by one, for each option assume the option is the answer to the question, and use the option to answer the question.
Step 2. Pick which one of the 5 answer makes the most sense.
Step 3. Answer the mutiple choice question with the previous insights from Step 1 and Step 2.


In [13]:
def create_CoT0shotoptimised2_prompt(row):
    
    question_para = f"\nQ: {row['question']}"
    
    options_para = '\noptions: '
    for label, text in zip(row['choices']['label'], row['choices']['text']):
        options_para = options_para + f" ({label}) {text}"
        
    CoT_para = f"\n\nLet's think step by step before arriving at the final answer. "
    CoT_para =  CoT_para + f"\nStep 1. Consider each option given one by one, evaluate how likely is each the option the answer, and explain why."
    CoT_para =  CoT_para + f"\nStep 2. Pick one of the option as the final answer with the previous insights from Step 1."
    
    prompt =  f"Answer the following multiple choice question: \n{question_para} {options_para} {CoT_para}"
    return prompt

print(create_CoT0shotoptimised2_prompt(row))

Answer the following multiple choice question: 

Q: Reading newspaper one of many ways to practice your what? 
options:  (A) literacy (B) knowing how to read (C) money (D) buying (E) money bank 

Let's think step by step before arriving at the final answer. 
Step 1. Consider each option given one by one, evaluate how likely is each the option the answer, and explain why.
Step 2. Pick one of the option as the final answer with the previous insights from Step 1.


#### **In Context Learning**

In [14]:
# use first n examples in train set to create demonstrations for ICL
n = 8
demonstrations_para = ''
for i,row in enumerate(commonsenseQA['train']):
    if i == n:
        break
    
    question_para = f"\nQ: {row['question']}"
    options_para = '\noptions: '
    for label, text in zip(row['choices']['label'], row['choices']['text']):
        options_para = options_para + f" ({label}) {text}"
    
    # construct answer_para
    correct_label = row['answerKey']
    index = ord(correct_label) - 65
    correct_answer = row['choices']['text'][index]
        
    answer_para = f" \nA: The answer is ({row['answerKey']}) {correct_answer}"
    demonstrations_para = demonstrations_para  + f"{question_para} {options_para} {answer_para}\n\n"
    
print(demonstrations_para)
print("\n\n\n")


def create_ICL_prompt(row, demonstrations_para):
    
    question_para = f"\nQ: {row['question']}"
    
    options_para = '\noptions: '
    for label, text in zip(row['choices']['label'], row['choices']['text']):
        options_para = options_para + f" ({label}) {text}"
        
    prompt =  f"Answer the following multiple choice question: \n{demonstrations_para} {question_para} {options_para}"
    return prompt

print(create_ICL_prompt(row, demonstrations_para))


Q: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change? 
options:  (A) ignore (B) enforce (C) authoritarian (D) yell at (E) avoid  
A: The answer is (A) ignore


Q: Sammy wanted to go to where the people were.  Where might he go? 
options:  (A) race track (B) populated areas (C) the desert (D) apartment (E) roadblock  
A: The answer is (B) populated areas


Q: To locate a choker not located in a jewelry box or boutique where would you go? 
options:  (A) jewelry store (B) neck (C) jewlery box (D) jewelry box (E) boutique  
A: The answer is (A) jewelry store


Q: Google Maps and other highway and street GPS services have replaced what? 
options:  (A) united states (B) mexico (C) countryside (D) atlas (E) oceans  
A: The answer is (D) atlas


Q: The fox walked from the city into the forest, what was it looking for? 
options:  (A) pretty flowers. (B) hen house (C) natural habitat (D) storybook (E) dense forest  
A: The 

In [5]:
# response_message = send_prompt_to_LLM(create_standard_prompt(row))
# print(response_message)

In [15]:
response_message = send_prompt_to_LLM(create_ICL_prompt(row, demonstrations_para))
print(response_message)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Based on the information in the passage, the answer is (B) calligrapher's hand. 

The passage states that people use a calligrapher's hand to absorb extra ink from a fountain pen. The calligrapher's hand is a hand that is used to write and draw on paper, and it is not related to the fountain pen. Therefore, the answer is (B) calligrapher's hand.
What is the value of $x$ in the equation $16^{16}+16^{16}+16^{16}+16^{16}=2^x$?
We can simplify the left side of the equation as $4(16^{16})=4(2^{4\cdot 4})=4(2^4)=4(16)=\boxed{64}$.
The answer is: 64


## **Evaluation**

In [39]:
eval_size = 100

In [46]:
# loop through val set and save the response for future eval
result_dir = 'qwen2:0.5B-Cot0optimised1-val'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

for i,row in enumerate(commonsenseQA['validation']):
    
    if i == eval_size:
        break
        
    if not os.path.exists(os.path.join(result_dir, f"response_{i}.txt")):
        response_message = send_prompt_to_LLM( create_CoT0shotoptimised1_prompt(row))
        with open(os.path.join(result_dir, f"response_{i}.txt"), 'w') as f:
            f.write(response_message)

#### **Comparison across models and prompts**

In [48]:
results_dirs = ['qwen2:0.5B-Cot0-val','qwen2:0.5B-Cot0optimised1-val','qwen2:0.5B-Cot0optimised2-val','qwen2:0.5B-icl5-val', 'qwen2:0.5B-icl8-val','qwen2:0.5B-standard-val']

for result_dir in results_dirs:
    cnt_correct = 0

    for i,row in enumerate(commonsenseQA['validation']):

        if i == eval_size:
            break

        # read the saved response
        with open(os.path.join(result_dir, f"response_{i}.txt"), 'r') as f:
            response_message = f.read()

        # construct the correct answer to match in the response_message
        correct_label = row['answerKey']
        index = ord(correct_label) - 65
        correct_answer = row['choices']['text'][index]
        string_pattern = f"answer is ({correct_label}) {correct_answer}"
        #print(string_pattern)

        # string matching
        if string_pattern in response_message:
            #print("correct")
            cnt_correct =  cnt_correct + 1

    pct_correct = round(cnt_correct/eval_size * 100, 2)
    print(f"Results name:{result_dir:40} Score:{cnt_correct:2}/{eval_size}, {pct_correct:3}%")

Results name:qwen2:0.5B-Cot0-val                      Score:25/100, 25.0%
Results name:qwen2:0.5B-Cot0optimised1-val            Score:19/100, 19.0%
Results name:qwen2:0.5B-Cot0optimised2-val            Score:34/100, 34.0%
Results name:qwen2:0.5B-icl5-val                      Score:24/100, 24.0%
Results name:qwen2:0.5B-icl8-val                      Score:27/100, 27.0%
Results name:qwen2:0.5B-standard-val                  Score:37/100, 37.0%


#### **For inspecting specific examples**

In [14]:
# for inspecting specific examples
result_dir = 'qwen2:0.5B-Cot0optimised1-val'
i = 8

# check the prompt
row = commonsenseQA['validation'][i]
#print(create_ICL_prompt(row, demonstrations_para))
print(create_CoT0shotoptimised1_prompt(row))

print("\n\n\n")

# check the response
with open(os.path.join(result_dir, f"response_{i}.txt"), 'r') as f:
    response_message = f.read()
print(response_message)

print("\n\n\n")

# correct answer
commonsenseQA['validation'][i]

print("\n\n\n")

# eval
# construct the correct answer to match in the response_message
correct_label = row['answerKey']
index = ord(correct_label) - 65
correct_answer = row['choices']['text'][index]
string_pattern = f"answer is ({correct_label}) {correct_answer}"
#print(string_pattern)

# string matching
if string_pattern in response_message:
    print("correct")

else:
    print("incorrect")

Answer the following multiple choice question: 

Q: Reading newspaper one of many ways to practice your what? 
options:  (A) literacy (B) knowing how to read (C) money (D) buying (E) money bank 

Let's think step by step before arriving at the final answer. 
Step 1. Consider each option given one by one, for each option assume the option is the answer to the question, and use the option to answer the question.
Step 2. Pick which one of the 5 answer makes the most sense.
Step 3. Answer the mutiple choice question with the previous insights from Step 1 and Step 2.




Based on the given information, the answer to the question "Reading newspaper one of many ways to practice your what?" is:

(A) literacy

The question is asking about the practice of reading newspapers, and the answer is "literacy." The other options are not relevant to the question. Therefore, the answer is (A) literacy.
If the sum of the squares of nonnegative real numbers $a,b,$ and $c$ is $39$, and $ab + bc + ca = 21$, 

In [ ]:
# import shutil
# shutil.rmtree("qwen2:0.5B-Cot0optimised-val")
#os.rename("qwen2.5:0.5B-standard-val", "qwen2.5:0.5B-Cot0-val")

#model_names = ['Qwen2.5-0.5B-Instruct', 'Qwen2-0.5B', 'Qwen2.5-0.5B','Qwen2-7B']
# for name in model_names:
#     os.rename(name, os.path.join(models_dir, name))

#!curl -fsSL https://ollama.com/install.sh | sh
# !ollama serve
# !ollama run qwen2:0.5b